In [1]:
import pandas as pd
sample_submission = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')
sample_submission

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


In [2]:
import pandas as pd
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import sys

def charger_dataset(fichier):
    """Charge le dataset avec gestion d'erreurs."""
    try:
        print("Chargement du dataset...")
        df = pd.read_csv(fichier)
        print(f"Dataset chargé avec succès: {len(df)} lignes")
        return df
    except Exception as e:
        print(f"Erreur lors du chargement du dataset: {str(e)}")
        sys.exit(1)

def charger_modele():
    """Charge le modèle avec gestion d'erreurs et timeout."""
    try:
        print("Initialisation du tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True
        )
        print("Tokenizer chargé avec succès")
        
        print("Chargement du modèle...")
        model = AutoModelForCausalLM.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True,
            device_map="auto",
            torch_dtype=torch.float16  # Utiliser la précision réduite
        )
        print("Modèle chargé avec succès")
        return tokenizer, model
    except Exception as e:
        print(f"Erreur lors du chargement du modèle: {str(e)}")
        sys.exit(1)

def calculer_perplexite(text, tokenizer, model, max_length=512):
    """Calcule la perplexité avec gestion des erreurs et limitation de longueur."""
    try:
        # Tronquer le texte si nécessaire
        tokens = tokenizer.tokenize(text)
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
            text = tokenizer.convert_tokens_to_string(tokens)
        
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            perplexite = torch.exp(loss)
        return perplexite.item()
    except Exception as e:
        print(f"Erreur lors du calcul de la perplexité: {str(e)}")
        return float('inf')

def algorithme_genetique_simple(dataset, tokenizer, model, taille_population=5, generations=5):
    """Version simplifiée de l'algorithme pour le débogage."""
    try:
        # Prendre juste une partie du texte pour le test
        texte = dataset["text"].iloc[0]
        mots = texte.split()[:50]  # Limiter à 50 mots pour le test
        print(f"Nombre de mots utilisés: {len(mots)}")
        
        # Générer une petite population initiale
        population = []
        for i in range(taille_population):
            individu = mots[:]
            random.shuffle(individu)
            population.append(individu)
            print(f"Individu {i+1} généré")
        
        meilleure_perplexite = float('inf')
        
        # Boucle principale avec timeout
        debut = time.time()
        timeout = 300  # 5 minutes maximum
        
        for generation in range(generations):
            print(f"\nGénération {generation + 1}/{generations}")
            
            # Vérifier le timeout
            if time.time() - debut > timeout:
                print("Timeout atteint")
                break
            
            # Évaluer chaque individu
            for i, individu in enumerate(population):
                texte = " ".join(individu)
                perplexite = calculer_perplexite(texte, tokenizer, model)
                print(f"Individu {i+1}: Perplexité = {perplexite}")
                
                if perplexite < meilleure_perplexite:
                    meilleure_perplexite = perplexite
                    meilleur_individu = individu[:]
            
            print(f"Meilleure perplexité de la génération: {meilleure_perplexite}")
            
            # Créer nouvelle génération
            nouvelle_population = []
            for _ in range(taille_population):
                # Sélection aléatoire simple pour le test
                parent1 = random.choice(population)
                parent2 = random.choice(population)
                
                # Croisement simple
                point = random.randint(0, len(mots)-1)
                enfant = parent1[:point] + [mot for mot in parent2 if mot not in parent1[:point]]
                nouvelle_population.append(enfant)
            
            population = nouvelle_population
            
        return meilleur_individu, meilleure_perplexite
        
    except Exception as e:
        print(f"Erreur dans l'algorithme génétique: {str(e)}")
        return None, float('inf')

# Test principal avec gestion d'erreurs
def main():
    try:
        print("Début du programme")
        
        # Charger le dataset
        fichier = "/kaggle/input/santa-2024/sample_submission.csv"
        dataset = charger_dataset(fichier)
        
        # Charger le modèle
        print("\nChargement du modèle et tokenizer...")
        tokenizer, model = charger_modele()
        
        # Exécuter l'algorithme simplifié
        print("\nDémarrage de l'algorithme génétique...")
        meilleur_individu, meilleure_perplexite = algorithme_genetique_simple(
            dataset, tokenizer, model
        )
        
        if meilleur_individu is not None:
            print("\nRésultats finaux:")
            print("Meilleure réorganisation:", " ".join(meilleur_individu))
            print("Perplexité minimale:", meilleure_perplexite)
        
    except KeyboardInterrupt:
        print("\nProgramme interrompu par l'utilisateur")
    except Exception as e:
        print(f"\nErreur générale: {str(e)}")

if __name__ == "__main__":
    main()

Début du programme
Chargement du dataset...
Dataset chargé avec succès: 6 lignes

Chargement du modèle et tokenizer...
Initialisation du tokenizer...
Tokenizer chargé avec succès
Chargement du modèle...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Modèle chargé avec succès

Démarrage de l'algorithme génétique...
Nombre de mots utilisés: 10
Individu 1 généré
Individu 2 généré
Individu 3 généré
Individu 4 généré
Individu 5 généré

Génération 1/5
Individu 1: Perplexité = 2687.506103515625
Individu 2: Perplexité = 4887.08544921875
Individu 3: Perplexité = 1631.4600830078125
Individu 4: Perplexité = 5758.443359375
Individu 5: Perplexité = 2336.01611328125
Meilleure perplexité de la génération: 1631.4600830078125

Génération 2/5
Individu 1: Perplexité = 1631.4600830078125
Individu 2: Perplexité = 1631.4600830078125
Individu 3: Perplexité = 2687.506103515625
Individu 4: Perplexité = 2336.01611328125
Individu 5: Perplexité = 5116.30029296875
Meilleure perplexité de la génération: 1631.4600830078125

Génération 3/5
Individu 1: Perplexité = 1636.6243896484375
Individu 2: Perplexité = 2336.01611328125
Individu 3: Perplexité = 2098.802490234375
Individu 4: Perplexité = 2671.268798828125
Individu 5: Perplexité = 10105.2841796875
Meilleure pe

**Amélioration en utilisant de local search, intensification et diversification :**

In [3]:
import pandas as pd
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import sys
import numpy as np
from collections import deque

def charger_dataset(fichier):
    """Charge le dataset avec gestion d'erreurs."""
    try:
        print("Chargement du dataset...")
        df = pd.read_csv(fichier)
        print(f"Dataset chargé avec succès: {len(df)} lignes")
        return df
    except Exception as e:
        print(f"Erreur lors du chargement du dataset: {str(e)}")
        sys.exit(1)

def charger_modele():
    """Charge le modèle avec gestion d'erreurs et timeout."""
    try:
        print("Initialisation du tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True
        )
        print("Tokenizer chargé avec succès")
        
        print("Chargement du modèle...")
        model = AutoModelForCausalLM.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
        print("Modèle chargé avec succès")
        return tokenizer, model
    except Exception as e:
        print(f"Erreur lors du chargement du modèle: {str(e)}")
        sys.exit(1)

def calculer_perplexite(text, tokenizer, model, max_length=512):
    """Calcule la perplexité avec gestion des erreurs."""
    try:
        tokens = tokenizer.tokenize(text)
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
            text = tokenizer.convert_tokens_to_string(tokens)
        
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            perplexite = torch.exp(loss)
        return perplexite.item()
    except Exception as e:
        print(f"Erreur lors du calcul de la perplexité: {str(e)}")
        return float('inf')

def intensification_locale(solution, tokenizer, model, iterations=10):
    """Recherche locale intensive avec mémoire."""
    meilleure_solution = solution[:]
    meilleure_perplexite = calculer_perplexite(" ".join(solution), tokenizer, model)
    memoire = set()
    
    for _ in range(iterations):
        # Générer plusieurs voisins
        for _ in range(3):  # Essayer 3 voisins différents
            nouvelle_solution = meilleure_solution[:]
            # Appliquer 2-3 permutations
            for _ in range(random.randint(2, 3)):
                i, j = random.sample(range(len(nouvelle_solution)), 2)
                nouvelle_solution[i], nouvelle_solution[j] = nouvelle_solution[j], nouvelle_solution[i]
            
            # Vérifier si cette solution a déjà été testée
            solution_str = " ".join(nouvelle_solution)
            if solution_str not in memoire:
                memoire.add(solution_str)
                nouvelle_perplexite = calculer_perplexite(solution_str, tokenizer, model)
                
                if nouvelle_perplexite < meilleure_perplexite:
                    meilleure_perplexite = nouvelle_perplexite
                    meilleure_solution = nouvelle_solution[:]
                    print(f"Amélioration trouvée! Nouvelle perplexité: {meilleure_perplexite}")
    
    return meilleure_solution, meilleure_perplexite

def diversification(population, nb_mots):
    """Diversification de la population."""
    population_diversifiee = []
    
    # Conserver les 2 meilleurs individus
    population_diversifiee.extend(population[:2])
    
    # Ajouter des solutions avec des perturbations fortes
    while len(population_diversifiee) < len(population):
        nouvel_individu = random.choice(population)[:]
        # Appliquer plusieurs permutations aléatoires
        nb_permutations = random.randint(5, 10)
        for _ in range(nb_permutations):
            i, j = random.sample(range(nb_mots), 2)
            nouvel_individu[i], nouvel_individu[j] = nouvel_individu[j], nouvel_individu[i]
        population_diversifiee.append(nouvel_individu)
    
    return population_diversifiee

def algorithme_genetique_optimise(dataset, tokenizer, model, taille_population=10, generations=10):
    """Algorithme génétique avec intensification et diversification."""
    try:
        print("\nInitialisation de l'algorithme génétique optimisé...")
        texte = dataset["text"].iloc[0]
        mots = texte.split()[:100]  # Augmenté à 100 mots
        nb_mots = len(mots)
        print(f"Nombre de mots utilisés: {nb_mots}")
        
        # Population initiale
        population = []
        meilleure_perplexite = float('inf')
        meilleur_individu = None
        meilleurs_scores = deque(maxlen=5)  # Pour détecter la stagnation
        
        print("Génération de la population initiale...")
        for i in range(taille_population):
            individu = mots[:]
            random.shuffle(individu)
            population.append(individu)
            print(f"Individu {i+1} généré")
        
        # Boucle principale
        debut = time.time()
        timeout = 600  # 10 minutes
        stagnation_counter = 0
        
        for generation in range(generations):
            print(f"\nGénération {generation + 1}/{generations}")
            
            if time.time() - debut > timeout:
                print("Timeout atteint")
                break
            
            # Évaluation et tri de la population
            population_evaluee = []
            for individu in population:
                perplexite = calculer_perplexite(" ".join(individu), tokenizer, model)
                population_evaluee.append((individu, perplexite))
            
            population_evaluee.sort(key=lambda x: x[1])
            population = [ind for ind, _ in population_evaluee]
            perplexites = [perp for _, perp in population_evaluee]
            
            # Mise à jour du meilleur
            if perplexites[0] < meilleure_perplexite:
                meilleure_perplexite = perplexites[0]
                meilleur_individu = population[0][:]
                print(f"Nouvelle meilleure perplexité: {meilleure_perplexite}")
            
            # Détection de stagnation
            meilleurs_scores.append(meilleure_perplexite)
            if len(meilleurs_scores) >= 5:
                if max(meilleurs_scores) - min(meilleurs_scores) < 0.1:
                    stagnation_counter += 1
                else:
                    stagnation_counter = 0
            
            # Phase d'intensification
            if generation % 2 == 0:  # Toutes les 2 générations
                print("Application de l'intensification...")
                population[0], nouvelle_perplexite = intensification_locale(population[0], tokenizer, model)
                if nouvelle_perplexite < meilleure_perplexite:
                    meilleure_perplexite = nouvelle_perplexite
                    meilleur_individu = population[0][:]
            
            # Phase de diversification si stagnation
            if stagnation_counter >= 2:
                print("Stagnation détectée, application de la diversification...")
                population = diversification(population, nb_mots)
                stagnation_counter = 0
            
            # Création nouvelle génération
            nouvelle_population = [population[0]]  # Élitisme
            while len(nouvelle_population) < taille_population:
                # Sélection par tournoi
                candidats = random.sample(population[:5], 2)  # Sélection parmi les 5 meilleurs
                parent1 = min(candidats, key=lambda x: calculer_perplexite(" ".join(x), tokenizer, model))
                candidats = random.sample(population[:5], 2)
                parent2 = min(candidats, key=lambda x: calculer_perplexite(" ".join(x), tokenizer, model))
                
                # Croisement amélioré
                point1, point2 = sorted(random.sample(range(nb_mots), 2))
                enfant = parent1[point1:point2]
                reste = [mot for mot in parent2 if mot not in enfant]
                enfant = reste[:point1] + enfant + reste[point1:]
                
                # Mutation adaptative
                if random.random() < 0.2:  # 20% de chance de mutation
                    nb_mutations = random.randint(1, 3)
                    for _ in range(nb_mutations):
                        i, j = random.sample(range(nb_mots), 2)
                        enfant[i], enfant[j] = enfant[j], enfant[i]
                
                nouvelle_population.append(enfant)
            
            population = nouvelle_population
        
        # Intensification finale sur le meilleur
        print("\nApplication de l'intensification finale...")
        meilleur_individu, meilleure_perplexite = intensification_locale(
            meilleur_individu,
            tokenizer,
            model,
            iterations=20  # Plus d'itérations pour l'intensification finale
        )
        
        return meilleur_individu, meilleure_perplexite
        
    except Exception as e:
        print(f"Erreur dans l'algorithme génétique: {str(e)}")
        return None, float('inf')

def main():
    try:
        print("Début du programme")
        fichier = "/kaggle/input/santa-2024/sample_submission.csv"
        dataset = charger_dataset(fichier)
        tokenizer, model = charger_modele()
        
        print("\nDémarrage de l'algorithme génétique optimisé...")
        meilleur_individu, meilleure_perplexite = algorithme_genetique_optimise(
            dataset, tokenizer, model
        )
        
        if meilleur_individu is not None:
            print("\nRésultats finaux:")
            print("Meilleure réorganisation:", " ".join(meilleur_individu))
            print("Perplexité minimale:", meilleure_perplexite)
        
    except KeyboardInterrupt:
        print("\nProgramme interrompu par l'utilisateur")
    except Exception as e:
        print(f"\nErreur générale: {str(e)}")

if __name__ == "__main__":
    main()

Début du programme
Chargement du dataset...
Dataset chargé avec succès: 6 lignes
Initialisation du tokenizer...
Tokenizer chargé avec succès
Chargement du modèle...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Modèle chargé avec succès

Démarrage de l'algorithme génétique optimisé...

Initialisation de l'algorithme génétique optimisé...
Nombre de mots utilisés: 10
Génération de la population initiale...
Individu 1 généré
Individu 2 généré
Individu 3 généré
Individu 4 généré
Individu 5 généré
Individu 6 généré
Individu 7 généré
Individu 8 généré
Individu 9 généré
Individu 10 généré

Génération 1/10
Nouvelle meilleure perplexité: 1986.6278076171875
Application de l'intensification...
Amélioration trouvée! Nouvelle perplexité: 1878.879150390625
Amélioration trouvée! Nouvelle perplexité: 1838.3741455078125
Amélioration trouvée! Nouvelle perplexité: 1682.903076171875

Génération 2/10
Nouvelle meilleure perplexité: 1622.746337890625

Génération 3/10
Nouvelle meilleure perplexité: 1340.7545166015625
Application de l'intensification...

Génération 4/10
Nouvelle meilleure perplexité: 1269.801025390625

Génération 5/10
Application de l'intensification...
Amélioration trouvée! Nouvelle perplexité: 1177

**Amélioration en utilisant de recuit_simule, intensification et diversification :**

In [4]:
import pandas as pd
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import sys
import numpy as np
import math
from collections import deque

def charger_dataset(fichier):
    """Charge le dataset avec gestion d'erreurs."""
    try:
        print("Chargement du dataset...")
        df = pd.read_csv(fichier)
        print(f"Dataset chargé avec succès: {len(df)} lignes")
        return df
    except Exception as e:
        print(f"Erreur lors du chargement du dataset: {str(e)}")
        sys.exit(1)

def charger_modele():
    """Charge le modèle avec gestion d'erreurs et timeout."""
    try:
        print("Initialisation du tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True
        )
        print("Tokenizer chargé avec succès")
        
        print("Chargement du modèle...")
        model = AutoModelForCausalLM.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
        print("Modèle chargé avec succès")
        return tokenizer, model
    except Exception as e:
        print(f"Erreur lors du chargement du modèle: {str(e)}")
        sys.exit(1)

def calculer_perplexite(text, tokenizer, model, max_length=512):
    """Calcule la perplexité avec gestion des erreurs."""
    try:
        tokens = tokenizer.tokenize(text)
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
            text = tokenizer.convert_tokens_to_string(tokens)
        
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            perplexite = torch.exp(loss)
        return perplexite.item()
    except Exception as e:
        print(f"Erreur lors du calcul de la perplexité: {str(e)}")
        return float('inf')

def intensification_locale(solution, tokenizer, model, iterations=10):
    """Recherche locale intensive avec mémoire."""
    meilleure_solution = solution[:]
    meilleure_perplexite = calculer_perplexite(" ".join(solution), tokenizer, model)
    memoire = set()
    
    for _ in range(iterations):
        for _ in range(3):  # Essayer 3 voisins différents
            nouvelle_solution = meilleure_solution[:]
            # Appliquer 2-3 permutations
            for _ in range(random.randint(2, 3)):
                i, j = random.sample(range(len(nouvelle_solution)), 2)
                nouvelle_solution[i], nouvelle_solution[j] = nouvelle_solution[j], nouvelle_solution[i]
            
            solution_str = " ".join(nouvelle_solution)
            if solution_str not in memoire:
                memoire.add(solution_str)
                nouvelle_perplexite = calculer_perplexite(solution_str, tokenizer, model)
                
                if nouvelle_perplexite < meilleure_perplexite:
                    meilleure_perplexite = nouvelle_perplexite
                    meilleure_solution = nouvelle_solution[:]
                    print(f"Amélioration trouvée! Nouvelle perplexité: {meilleure_perplexite}")
    
    return meilleure_solution, meilleure_perplexite

def recuit_simule(solution_initiale, tokenizer, model, temperature_initiale=100, alpha=0.95, 
                  iterations_par_palier=20, min_temperature=0.01):
    """
    Optimise la solution avec l'algorithme du recuit simulé.
    
    Args:
        solution_initiale: Liste de mots à optimiser
        tokenizer: Tokenizer pour le calcul de la perplexité
        model: Modèle pour le calcul de la perplexité
        temperature_initiale: Température initiale du système
        alpha: Facteur de refroidissement (entre 0 et 1)
        iterations_par_palier: Nombre d'itérations à chaque température
        min_temperature: Température minimale pour arrêter l'algorithme
        
    Returns:
        Tuple (meilleure_solution, meilleure_perplexite)
    """
    def accepter_solution(delta_perplexite, temperature):
        if delta_perplexite < 0:  # Si la nouvelle solution est meilleure
            return True
        else:
            probabilite = math.exp(-delta_perplexite / temperature)
            return random.random() < probabilite
    
    solution_courante = solution_initiale[:]
    meilleure_solution = solution_courante[:]
    perplexite_courante = calculer_perplexite(" ".join(solution_courante), tokenizer, model)
    meilleure_perplexite = perplexite_courante
    
    temperature = temperature_initiale
    nb_mots = len(solution_initiale)
    memoire = set()  # Pour éviter de revisiter les mêmes solutions
    stagnation_counter = 0
    debut = time.time()
    timeout = 600  # 10 minutes
    
    print("Début du recuit simulé...")
    while temperature > min_temperature:
        # Vérifier le timeout
        if time.time() - debut > timeout:
            print("Timeout atteint")
            break
            
        amelioration_trouvee = False
        
        for _ in range(iterations_par_palier):
            # Générer une nouvelle solution voisine
            nouvelle_solution = solution_courante[:]
            
            # Perturbation adaptative basée sur la température
            nb_permutations = max(1, int(3 * temperature / temperature_initiale))
            positions = random.sample(range(nb_mots), 2 * nb_permutations)
            
            for i in range(0, len(positions), 2):
                nouvelle_solution[positions[i]], nouvelle_solution[positions[i+1]] = \
                    nouvelle_solution[positions[i+1]], nouvelle_solution[positions[i]]
            
            # Vérifier si cette solution a déjà été testée
            solution_str = " ".join(nouvelle_solution)
            if solution_str in memoire:
                continue
            
            memoire.add(solution_str)
            nouvelle_perplexite = calculer_perplexite(solution_str, tokenizer, model)
            delta = nouvelle_perplexite - perplexite_courante
            
            if accepter_solution(delta, temperature):
                solution_courante = nouvelle_solution
                perplexite_courante = nouvelle_perplexite
                amelioration_trouvee = True
                
                if nouvelle_perplexite < meilleure_perplexite:
                    meilleure_solution = nouvelle_solution[:]
                    meilleure_perplexite = nouvelle_perplexite
                    print(f"Nouvelle meilleure perplexité: {meilleure_perplexite} (T={temperature:.2f})")
                    stagnation_counter = 0
                
        if not amelioration_trouvee:
            stagnation_counter += 1
        
        # Mécanisme de redémarrage en cas de stagnation
        if stagnation_counter >= 5:
            print("Stagnation détectée, redémarrage partiel...")
            # Perturber fortement la solution courante
            positions = random.sample(range(nb_mots), nb_mots // 3)
            for i in range(0, len(positions), 2):
                if i + 1 < len(positions):
                    solution_courante[positions[i]], solution_courante[positions[i+1]] = \
                        solution_courante[positions[i+1]], solution_courante[positions[i]]
            perplexite_courante = calculer_perplexite(" ".join(solution_courante), tokenizer, model)
            temperature = temperature_initiale * 0.5  # Redémarrer avec une température plus basse
            stagnation_counter = 0
        
        # Refroidissement
        temperature *= alpha
        print(f"Température actuelle: {temperature:.2f}")
    
    # Intensification finale
    print("\nApplication de l'intensification finale...")
    meilleure_solution, meilleure_perplexite = intensification_locale(
        meilleure_solution,
        tokenizer,
        model,
        iterations=20  # Plus d'itérations pour l'intensification finale
    )
    
    return meilleure_solution, meilleure_perplexite

def main():
    try:
        print("Début du programme")
        fichier = "/kaggle/input/santa-2024/sample_submission.csv"
        dataset = charger_dataset(fichier)
        tokenizer, model = charger_modele()
        
        print("\nDémarrage du recuit simulé...")
        texte = dataset["text"].iloc[0]
        mots = texte.split()[:100]  # On utilise les 100 premiers mots
        solution_initiale = mots[:]
        random.shuffle(solution_initiale)  # Solution initiale aléatoire
        
        # Premier passage avec recuit simulé
        meilleur_individu, meilleure_perplexite = recuit_simule(
            solution_initiale,
            tokenizer,
            model,
            temperature_initiale=100,
            alpha=0.95,
            iterations_par_palier=20
        )
        
        # Deuxième passage avec température plus basse pour affiner
        meilleur_individu, meilleure_perplexite = recuit_simule(
            meilleur_individu,
            tokenizer,
            model,
            temperature_initiale=50,
            alpha=0.98,
            iterations_par_palier=30
        )
        
        if meilleur_individu is not None:
            print("\nRésultats finaux:")
            print("Meilleure réorganisation:", " ".join(meilleur_individu))
            print("Perplexité minimale:", meilleure_perplexite)
            
            # Sauvegarder les résultats
            with open("resultats_recuit_simule.txt", "w", encoding="utf-8") as f:
                f.write("Meilleure réorganisation: " + " ".join(meilleur_individu) + "\n")
                f.write(f"Perplexité minimale: {meilleure_perplexite}\n")
        
    except KeyboardInterrupt:
        print("\nProgramme interrompu par l'utilisateur")
    except Exception as e:
        print(f"\nErreur générale: {str(e)}")

if __name__ == "__main__":
    main()

Début du programme
Chargement du dataset...
Dataset chargé avec succès: 6 lignes
Initialisation du tokenizer...
Tokenizer chargé avec succès
Chargement du modèle...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Modèle chargé avec succès

Démarrage du recuit simulé...
Début du recuit simulé...
Nouvelle meilleure perplexité: 1526.1688232421875 (T=100.00)
Nouvelle meilleure perplexité: 1402.73828125 (T=100.00)
Température actuelle: 95.00
Nouvelle meilleure perplexité: 1339.191650390625 (T=95.00)
Nouvelle meilleure perplexité: 1219.1839599609375 (T=95.00)
Température actuelle: 90.25
Nouvelle meilleure perplexité: 776.8199462890625 (T=90.25)
Température actuelle: 85.74
Nouvelle meilleure perplexité: 505.50396728515625 (T=85.74)
Température actuelle: 81.45
Température actuelle: 77.38
Température actuelle: 73.51
Température actuelle: 69.83
Température actuelle: 66.34
Température actuelle: 63.02
Température actuelle: 59.87
Température actuelle: 56.88
Température actuelle: 54.04
Température actuelle: 51.33
Température actuelle: 48.77
Température actuelle: 46.33
Température actuelle: 44.01
Température actuelle: 41.81
Température actuelle: 39.72
Température actuelle: 37.74
Température actuelle: 35.85
Te

**Traitemant de tous les textes du dataset :**

In [6]:
import pandas as pd
import random
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import sys
import numpy as np
import math
from collections import deque

def charger_dataset(fichier):
    """Charge le dataset avec gestion d'erreurs."""
    try:
        print("Chargement du dataset...")
        df = pd.read_csv(fichier)
        print(f"Dataset chargé avec succès: {len(df)} lignes")
        return df
    except Exception as e:
        print(f"Erreur lors du chargement du dataset: {str(e)}")
        sys.exit(1)

def charger_modele():
    """Charge le modèle avec gestion d'erreurs et timeout."""
    try:
        print("Initialisation du tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True
        )
        print("Tokenizer chargé avec succès")
        
        print("Chargement du modèle...")
        model = AutoModelForCausalLM.from_pretrained(
            "/kaggle/input/gemma-2/transformers/gemma-2-9b/2",
            local_files_only=True,
            device_map="auto",
            torch_dtype=torch.float16
        )
        print("Modèle chargé avec succès")
        return tokenizer, model
    except Exception as e:
        print(f"Erreur lors du chargement du modèle: {str(e)}")
        sys.exit(1)

def calculer_perplexite(text, tokenizer, model, max_length=512):
    """Calcule la perplexité avec gestion des erreurs."""
    try:
        tokens = tokenizer.tokenize(text)
        if len(tokens) > max_length:
            tokens = tokens[:max_length]
            text = tokenizer.convert_tokens_to_string(tokens)
        
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
        with torch.no_grad():
            outputs = model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            perplexite = torch.exp(loss)
        return perplexite.item()
    except Exception as e:
        print(f"Erreur lors du calcul de la perplexité: {str(e)}")
        return float('inf')

def intensification_locale(solution, tokenizer, model, iterations=10):
    """Recherche locale intensive avec mémoire."""
    meilleure_solution = solution[:]
    meilleure_perplexite = calculer_perplexite(" ".join(solution), tokenizer, model)
    memoire = set()
    
    for _ in range(iterations):
        for _ in range(3):  # Essayer 3 voisins différents
            nouvelle_solution = meilleure_solution[:]
            # Appliquer 2-3 permutations
            for _ in range(random.randint(2, 3)):
                i, j = random.sample(range(len(nouvelle_solution)), 2)
                nouvelle_solution[i], nouvelle_solution[j] = nouvelle_solution[j], nouvelle_solution[i]
            
            solution_str = " ".join(nouvelle_solution)
            if solution_str not in memoire:
                memoire.add(solution_str)
                nouvelle_perplexite = calculer_perplexite(solution_str, tokenizer, model)
                
                if nouvelle_perplexite < meilleure_perplexite:
                    meilleure_perplexite = nouvelle_perplexite
                    meilleure_solution = nouvelle_solution[:]
                    print(f"Amélioration trouvée! Nouvelle perplexité: {meilleure_perplexite}")
    
    return meilleure_solution, meilleure_perplexite

def recuit_simule(solution_initiale, tokenizer, model, temperature_initiale=100, alpha=0.95, 
                  iterations_par_palier=20, min_temperature=0.01):
    """
    Optimise la solution avec l'algorithme du recuit simulé.
    
    Args:
        solution_initiale: Liste de mots à optimiser
        tokenizer: Tokenizer pour le calcul de la perplexité
        model: Modèle pour le calcul de la perplexité
        temperature_initiale: Température initiale du système
        alpha: Facteur de refroidissement (entre 0 et 1)
        iterations_par_palier: Nombre d'itérations à chaque température
        min_temperature: Température minimale pour arrêter l'algorithme
        
    Returns:
        Tuple (meilleure_solution, meilleure_perplexite)
    """
    def accepter_solution(delta_perplexite, temperature):
        if delta_perplexite < 0:  # Si la nouvelle solution est meilleure
            return True
        else:
            probabilite = math.exp(-delta_perplexite / temperature)
            return random.random() < probabilite
    
    solution_courante = solution_initiale[:]
    meilleure_solution = solution_courante[:]
    perplexite_courante = calculer_perplexite(" ".join(solution_courante), tokenizer, model)
    meilleure_perplexite = perplexite_courante
    
    temperature = temperature_initiale
    nb_mots = len(solution_initiale)
    memoire = set()  # Pour éviter de revisiter les mêmes solutions
    stagnation_counter = 0
    debut = time.time()
    timeout = 600  # 10 minutes
    
    print("Début du recuit simulé...")
    while temperature > min_temperature:
        # Vérifier le timeout
        if time.time() - debut > timeout:
            print("Timeout atteint")
            break
            
        amelioration_trouvee = False
        
        for _ in range(iterations_par_palier):
            # Générer une nouvelle solution voisine
            nouvelle_solution = solution_courante[:]
            
            # Perturbation adaptative basée sur la température
            nb_permutations = max(1, int(3 * temperature / temperature_initiale))
            positions = random.sample(range(nb_mots), 2 * nb_permutations)
            
            for i in range(0, len(positions), 2):
                nouvelle_solution[positions[i]], nouvelle_solution[positions[i+1]] = \
                    nouvelle_solution[positions[i+1]], nouvelle_solution[positions[i]]
            
            # Vérifier si cette solution a déjà été testée
            solution_str = " ".join(nouvelle_solution)
            if solution_str in memoire:
                continue
            
            memoire.add(solution_str)
            nouvelle_perplexite = calculer_perplexite(solution_str, tokenizer, model)
            delta = nouvelle_perplexite - perplexite_courante
            
            if accepter_solution(delta, temperature):
                solution_courante = nouvelle_solution
                perplexite_courante = nouvelle_perplexite
                amelioration_trouvee = True
                
                if nouvelle_perplexite < meilleure_perplexite:
                    meilleure_solution = nouvelle_solution[:]
                    meilleure_perplexite = nouvelle_perplexite
                    print(f"Nouvelle meilleure perplexité: {meilleure_perplexite} (T={temperature:.2f})")
                    stagnation_counter = 0
                
        if not amelioration_trouvee:
            stagnation_counter += 1
        
        # Mécanisme de redémarrage en cas de stagnation
        if stagnation_counter >= 5:
            print("Stagnation détectée, redémarrage partiel...")
            # Perturber fortement la solution courante
            positions = random.sample(range(nb_mots), nb_mots // 3)
            for i in range(0, len(positions), 2):
                if i + 1 < len(positions):
                    solution_courante[positions[i]], solution_courante[positions[i+1]] = \
                        solution_courante[positions[i+1]], solution_courante[positions[i]]
            perplexite_courante = calculer_perplexite(" ".join(solution_courante), tokenizer, model)
            temperature = temperature_initiale * 0.5  # Redémarrer avec une température plus basse
            stagnation_counter = 0
        
        # Refroidissement
        temperature *= alpha
        print(f"Température actuelle: {temperature:.2f}")
    
    # Intensification finale
    print("\nApplication de l'intensification finale...")
    meilleure_solution, meilleure_perplexite = intensification_locale(
        meilleure_solution,
        tokenizer,
        model,
        iterations=20  # Plus d'itérations pour l'intensification finale
    )
    
    return meilleure_solution, meilleure_perplexite

def main():
    try:
        print("Début du programme")
        fichier = "/kaggle/input/santa-2024/sample_submission.csv"
        dataset = charger_dataset(fichier)
        tokenizer, model = charger_modele()
        
        print("\nDémarrage du recuit simulé pour tous les textes...")
        
        resultats = []
        for index, row in dataset.iterrows():
            texte = row["text"]
            mots = texte.split()[:100]  # On utilise les 100 premiers mots pour simplifier
            solution_initiale = mots[:]
            random.shuffle(solution_initiale)  # Solution initiale aléatoire
            
            print(f"\nTraitement du texte {index + 1}/{len(dataset)}")
            
            # Premier passage avec recuit simulé
            meilleur_individu, meilleure_perplexite = recuit_simule(
                solution_initiale,
                tokenizer,
                model,
                temperature_initiale=100,
                alpha=0.95,
                iterations_par_palier=20
            )
            
            # Deuxième passage avec température plus basse pour affiner
            meilleur_individu, meilleure_perplexite = recuit_simule(
                meilleur_individu,
                tokenizer,
                model,
                temperature_initiale=50,
                alpha=0.98,
                iterations_par_palier=30
            )
            
            # Ajouter les résultats à la liste
            resultats.append({
                "index": index,
                "meilleure_reorganisation": " ".join(meilleur_individu),
                "perplexite_minimale": meilleure_perplexite
            })
        
        # Affichage des résultats dans le notebook
        print("\nAffichage des résultats:")
        df_resultats = pd.DataFrame(resultats)
        display(df_resultats)  # Utilisé pour afficher de manière structurée dans un notebook
        
        print("Tous les textes ont été traités et les résultats affichés avec succès.")
        
    except KeyboardInterrupt:
        print("\nProgramme interrompu par l'utilisateur")
    except Exception as e:
        print(f"\nErreur générale: {str(e)}")

if __name__ == "__main__":
    main()


Début du programme
Chargement du dataset...
Dataset chargé avec succès: 6 lignes
Initialisation du tokenizer...
Tokenizer chargé avec succès
Chargement du modèle...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Modèle chargé avec succès

Démarrage du recuit simulé pour tous les textes...

Traitement du texte 1/6
Début du recuit simulé...
Nouvelle meilleure perplexité: 4100.6865234375 (T=100.00)
Nouvelle meilleure perplexité: 2715.719482421875 (T=100.00)
Nouvelle meilleure perplexité: 943.501953125 (T=100.00)
Nouvelle meilleure perplexité: 767.3067626953125 (T=100.00)
Température actuelle: 95.00
Nouvelle meilleure perplexité: 701.9727172851562 (T=95.00)
Nouvelle meilleure perplexité: 665.5963134765625 (T=95.00)
Température actuelle: 90.25
Température actuelle: 85.74
Nouvelle meilleure perplexité: 648.751953125 (T=85.74)
Température actuelle: 81.45
Température actuelle: 77.38
Nouvelle meilleure perplexité: 628.158447265625 (T=77.38)
Nouvelle meilleure perplexité: 443.16583251953125 (T=77.38)
Température actuelle: 73.51
Température actuelle: 69.83
Température actuelle: 66.34
Température actuelle: 63.02
Température actuelle: 59.87
Température actuelle: 56.88
Température actuelle: 54.04
Températur

,index,meilleure_reorganisation,perplexite_minimale
0,0,reindeer mistletoe elf gingerbread ornament fa...,439.106964
1,1,reindeer sleep walk drive laugh jump bake give...,679.985413
2,2,sleigh yuletide grinch holiday carol cheer hol...,351.030212
3,3,sleigh unwrap the stocking magi holly yuletide...,399.646362
4,4,peppermint star of wonder and night season wis...,342.693787
5,5,from the walk eat cheer joy wish of unwrap lau...,365.245728


Tous les textes ont été traités et les résultats affichés avec succès.
